In [2]:
import pandas as pd
import os
import pydicom
import shutil
import random

In [3]:
df = pd.read_csv('qureai-cq500-boxes-updated.csv')
df.head()

,SOPInstanceUID,SeriesInstanceUID,StudyInstanceUID,data,labelName,number
0,1.2.276.0.7230010.3.1.4.296485376.1.1521713091...,1.2.276.0.7230010.3.1.3.296485376.1.1521713090...,1.2.276.0.7230010.3.1.2.296485376.1.1521713088...,"{'x': 320.95899, 'y': 235.81072999999998, 'wid...",Intraventricular,96
1,1.2.276.0.7230010.3.1.4.296485376.1.1521713091...,1.2.276.0.7230010.3.1.3.296485376.1.1521713090...,1.2.276.0.7230010.3.1.2.296485376.1.1521713088...,"{'x': 320.95899, 'y': 235.81072999999998, 'wid...",Intraventricular,96
2,1.2.276.0.7230010.3.1.4.296485376.1.1521713091...,1.2.276.0.7230010.3.1.3.296485376.1.1521713090...,1.2.276.0.7230010.3.1.2.296485376.1.1521713088...,"{'x': 320.95899, 'y': 235.81072999999998, 'wid...",Intraventricular,96
3,1.2.276.0.7230010.3.1.4.296485376.1.1521713091...,1.2.276.0.7230010.3.1.3.296485376.1.1521713090...,1.2.276.0.7230010.3.1.2.296485376.1.1521713088...,"{'x': 320.95899, 'y': 235.81072999999998, 'wid...",Intraventricular,96
4,1.2.276.0.7230010.3.1.4.296485376.1.1521713091...,1.2.276.0.7230010.3.1.3.296485376.1.1521713090...,1.2.276.0.7230010.3.1.2.296485376.1.1521713088...,"{'x': 318.53628, 'y': 244.69401000000002, 'wid...",Intraventricular,96


Find all scan numbers that deal with stroke and where we have bounding boxes

In [4]:
numbers = set()

for index, row in df.iterrows():
  numbers.add(row['number'] - 1)

print(len(numbers))

228


Copy all relevant files from external folder to working directory

In [ ]:
# Adjust to your path

ext_path = 'D:\MTP\CQ500'
int_path = './data/cq500'

def copy_file(source_file, destination_directory, destination_filename):
    try:
        shutil.copy(source_file, f"{destination_directory}/{destination_filename}")
    except FileNotFoundError:
        print("Source file not found.")
    except PermissionError:
        print("Permission error. Make sure you have the necessary permissions.")
    except shutil.Error as e:
        print(f"An error occurred while moving the file: {e}")

def extract_dicom_tags(dicom_file_path):
    # Load and read the DICOM file
    dicom_dataset = pydicom.dcmread(dicom_file_path)

    # Access DICOM tags as dictionary keys
    dicom_tags = {}
    for tag in dicom_dataset:
        tag_name = tag.name
        tag_value = tag.value
        dicom_tags[tag_name] = tag_value

    return dicom_tags

def process_file(file_path):
    # Do something with the file
    tags = extract_dicom_tags(file_path)
    if tags['SOP Instance UID'] in df['SOPInstanceUID'].unique():
        copy_file(file_path, './data/cq500', tags['SOP Instance UID'] + '.dcm')

def process_directory(directory_path):
    # Get a list of all files and directories inside the given directory
    for entry in os.listdir(directory_path):
        full_path = os.path.join(directory_path, entry)

        if os.path.isfile(full_path):
            # Process the file if it's a regular file
            process_file(full_path)

        elif os.path.isdir(full_path):
            # Recursively process the subdirectory if it's a directory
            process_directory(full_path)

for num in numbers:
  process_directory(f'D:\MTP\CQ500\CQ500CT{num} CQ500CT{num}')
          

## Create Tensorflow Object Detection CSV Files & Directory

### 1. Shuffle Files into respective directories

In [13]:
# Define the split ratio (e.g., 0.8 for 80% train, 20% validation)
split_ratio = 0.8

# Create a list of all image filenames
image_filenames = os.listdir('./data/cq500')
random.shuffle(image_filenames)  # Randomly shuffle the images

# Split the images into train and validation sets based on the split_ratio
train_size = int(len(image_filenames) * split_ratio)
train_images = image_filenames[:train_size]
test_images = image_filenames[train_size:]

# Copy files to respective directories
for image_filename in train_images:
    copy_file(f"./data/cq500/{image_filename}", "./scans/train", image_filename)
    
print("Train set size:", len(train_images))

for image_filename in test_images:
    copy_file(f"./data/cq500/{image_filename}", "./scans/test", image_filename)
    
print("Test set size:", len(test_images))

Train set size: 18718
Test set size: 4680


In [24]:
# Suppress the FutureWarning related to .append() in pandas
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

train_list = []
test_list = []

for index, row in df.iterrows():
    if isinstance(row['data'], str):
        bbox = eval(row['data'])
        if row['SOPInstanceUID'] + '.dcm' in train_images:
            train_list.append({
                'filename': row['SOPInstanceUID'] + '.dcm',
                'width': 512,
                'height': 512,
                'xmin': bbox['x'],
                'ymin': bbox['y'],
                'xmax': bbox['x'] + bbox['width'],
                'ymax': bbox['y'] + bbox['height'],
                'class': 'hemorrhage'    
            })
        if row['SOPInstanceUID'] + '.dcm' in test_images:
            test_list.append({
                'filename': row['SOPInstanceUID'] + '.dcm',
                'width': 512,
                'height': 512,
                'xmin': bbox['x'],
                'ymin': bbox['y'],
                'xmax': bbox['x'] + bbox['width'],
                'ymax': bbox['y'] + bbox['height'],
                'class': 'hemorrhage'    
            })

df_train = pd.DataFrame(train_list)
df_test = pd.DataFrame(test_list)
            
df_train.to_csv('scans/train_labels.csv', index = False, encoding='utf-8')
df_test.to_csv('scans/test_labels.csv', index = False, encoding='utf-8')

print(f"Train labels size: {len(df_train)} ({len(df_train) / (len(df_train) + len(df_test))  * 100:.2f}%)")
print(f"Test labels size: {len(df_test)} ({len(df_test) / (len(df_train) + len(df_test)) * 100:.2f}%)")

Train labels size: 31674 (79.88%)
Test labels size: 7980 (20.12%)
